In [4]:
import cv2
import numpy as np
import tensorflow as tf
import time

In [10]:
# === 1. Load model TFLite ===
interpreter = tf.lite.Interpreter(model_path="MobileFaceNet_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# === 2. Tiền xử lý ảnh ===
def preprocess_face(face, target_size=(112, 112)):
	face = cv2.resize(face, target_size)
	face = face.astype("float32") / 127.5 - 1
	return face

# === 3. Trích embedding ===
def get_embedding(face_rgb):
	face = preprocess_face(face_rgb)
	face = np.expand_dims(face, axis=0)
	interpreter.set_tensor(input_details[0]['index'], face)
	interpreter.invoke()
	embedding = interpreter.get_tensor(output_details[0]['index'])[0]
	return embedding

# === 4. So sánh cosine ===
def cosine_similarity(vec1, vec2):
	return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# === 5. Load embedding từ known.jpg ===
known_img = cv2.imread("known.jpg")
known_img = cv2.cvtColor(known_img, cv2.COLOR_BGR2RGB)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
faces = face_cascade.detectMultiScale(known_img, 1.3, 5)
if len(faces) == 0:
	raise ValueError("❌ Không tìm thấy khuôn mặt trong known.jpg")
(x, y, w, h) = faces[0]
known_face = known_img[y:y+h, x:x+w]
known_embedding = get_embedding(known_face)

# === 6. Mở camera và nhận diện live ===
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)

threshold = 0.5
detect_interval = 3
frame_id = 0
detected_faces = []
text_labels = []

print("🎥 Đang nhận diện khuôn mặt. Nhấn 'q' để thoát.")

while True:
	ret, frame = cap.read()
	if not ret:
		break

	frame_id += 1
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

	if frame_id % detect_interval == 0:
		detected_faces.clear()
		text_labels.clear()
		faces = face_cascade.detectMultiScale(gray, 1.3, 5)

		for (x, y, w, h) in faces:
			face_rgb = rgb[y:y+h, x:x+w]
			try:
				embedding = get_embedding(face_rgb)
				similarity = cosine_similarity(embedding, known_embedding)
				match = similarity > 1 - threshold
				text = f"Match: {match} ({similarity:.2f})"
				color = (0, 255, 0) if match else (0, 0, 255)
			except:
				text = "Face Error"
				color = (0, 255, 255)

			detected_faces.append(((x, y, w, h), color))
			text_labels.append((text, (x, y - 10), color))

	# Vẽ lại mỗi frame
	for (x, y, w, h), color in detected_faces:
		cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

	for text, pos, color in text_labels:
		cv2.putText(frame, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

	cv2.imshow("Smart Door Lock (Press 'q' to Quit)", frame)
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break

cap.release()
cv2.destroyAllWindows()


🎥 Đang nhận diện khuôn mặt. Nhấn 'q' để thoát.
